<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_1029bit_letter_cancellation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BIT 文字抹消課題
* filename: 2022_1029bit_letter_cancellation.ipynb
* date: 2022_1029
* author: 浅川伸一


In [ ]:
# このセルは 2 回実行しないといけないかも知れません
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname --upgrade > /dev/null 2>&1
    !git clone https://github.com/ShinAsakawa/bit.git
    import bit

isColab = bit.isColab
HOME = bit.HOME

if isColab:
    # 2022_0916 現在 PIL のバージョンが古く truetype フォント
    # の表示に不具合が出るためバージョン 9.2.0 以上に更新する
    !pip install --upgrade Pillow

import torch
import PIL
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib    

# 1. 刺激の作成
## 1.1 画面分割の定義と対応するフォントの設定

* font が 14 種類あって，そのサイズが 5 種類だから，フォントは全部で 70 種類存在する。
* 色は 9 種類
* 記号は 49 種類ある
* 色 X 記号 X フォントの種類 =  9 X 49 X 14 = 6174
* したがって，一つの位置に 6174 だけ刺激が存在する。
* そうすると全ての位置に刺激が 6174 だけ存在するので，刺激情報 `stims` は `stims[split_{split}] ^[len(pos)]` だけだから


画面の分割によって，フォントサイズが異なるため，事前に登録しておく

In [ ]:
import os
from glob import glob
from PIL import ImageFont

noto_font_dir = 'fonts'
notofonts_fnames = glob(os.path.join(noto_font_dir,'*otf'))
notofonts = {fname.split('/')[-1].split('.')[0]:{'fname':fname} for fname in notofonts_fnames}
for fontname in notofonts.keys():
    notofonts[fontname]['data'] = ImageFont.truetype(notofonts[fontname]['fname'])
#notofonts;
symbols = bit.BIT(fontdata=notofonts).symbols     # 文字の登録
#print(symbols);

In [ ]:
#from PIL_util import colornames # 色名を定義
# `brown` を削除した
colornames = ['black', 'blue', 'cyan', 'green', 'magenta', 'orange', 'purple', 'red', 'yellow'] 

# フォント名の取得
fontnames = bit.get_notojp_fonts(verbose=False).keys()

class _params:
    def __init__(self,
                 splits:list=[2,3,4,5,6],
                 symbols:list=['<background>', '<line>', '★', 
                               'あ', 'い', 'う', 'え', 'お', 
                               'か', 'き', 'く', 'け', 'こ', 
                               'さ', 'し', 'す', 'せ', 'そ', 
                               'た', 'ち', 'つ', 'て', 'と', 
                               'な', 'に', 'ぬ', 'ね', 'の', 
                               'は', 'ひ', 'ふ', 'へ', 'ほ', 
                               'ま', 'み', 'む', 'め', 'も', 
                               'や', 'ゆ', 'よ', 
                               'ら', 'り', 'る', 'れ', 'ろ', 
                               'わ', 'を', 'ん'],
                 colornames:list=['black', 'blue', 'cyan', 
                                  'green', 'magenta', 'orange', 
                                  'purple', 'red', 'yellow'],
                 fontnames:list=fontnames,
                ):
        super().__init__()
        self.splits     = splits
        self.symbols    = symbols
        self.colornames = colornames
        self.fontnames  = fontnames

        
params = _params() 
#params = _params(splits=[1,3])
# for x in dir(params):
#     if not str(x).startswith('_'):
#         print(f'{x}:{eval(x)}')
splits = params.splits
print(splits)
        
        

In [ ]:
!wget https://ShinAsakawa.github.io/2022notebooks/PIL_util.py -O PIL_util.py
!wget https://ShinAsakawa.github.io/2022notebooks/bit_utils.py -O bit_utils.py

In [ ]:
#splits = [2,3,4,5,6]    # 画面の分割数
# #splits = [1, 3]            # 画面の分割数
from PIL_util import make_a_canvas as make_canvas  # PIL による画像とキャンバスの作成
from PIL_util import make_div_areas                # 領域を縦横に分割

# フォントサイズの計算
fontsizes = {}
#for split in params.splits: # 各条件ごと
for split in splits: # 各条件ごと
    areas = make_div_areas(div=split)  # 分割された領域
    fontsize = int((areas[0][3]) / 8 * 6) # 領域の大きさからフォントサイズを計算
    #fontsize = int((areas[0][3])/8 * 7) # 領域の大きさからフォントサイズを計算
    fontsizes[split] = {'split':split,
                        'size':fontsize}

fonts_info = {}      # フォント情報の登録
for key, fontsize in fontsizes.items():
    _fonts = bit.get_notojp_fonts(fontsize=fontsize['size'], verbose=False)
    for fontname, font in _fonts.items():
        font_entry = f"{fontsize['size']}_{fontname}"
        fonts_info[font_entry] = font

print(f'fontsizes:{fontsizes}')
print(f'len(symbols):{len(symbols)} symbols:{symbols}')
print(f'len(colornames):{len(colornames)} colornames:{colornames}')
print(f'len(fonts_info):{len(fonts_info)}')
print(f'fontnames:{fontnames}')

print(f'総刺激:{len(colornames) * len(symbols) * len(fontnames)} 種')

## 1.2 刺激作成条件に基づく刺激の作成

In [ ]:
# 訓練データセット，テストデータセットの作成
# 色 と フォント と 記号 との直交で全ての刺激を作成
# フォントサイズは，画面分割数に依存するため，各刺激画像で異なるが，それ以外の情報である，色，記号，および，フォントは
# 直交するので全組合わせを作成しておく。
# これを _stimset とする
_stimset = [] 
for font in fontnames:
    for color in colornames:
        for symbol in symbols:
            _stimset.append((symbol, color, font))
print(f'len(_stimset):{len(_stimset)}')

# 直上で作成した _stimset を並べ替えて，刺激画像上の各位置に描画する刺激を配置する。
# $\text{各位置}^{len(_stimset)}$ だけ，可能性があるが，数が多くなりすぎる
# そこで，_stimset を乱数を使って並べ替え，画像の各位置に現れる刺激として採用することにした。
# これにより，各画面分割数 `splits` を条件として，この条件毎に `len(_stimset)` 数だけの刺激画像が存在することとした。
stims = {f'split_{split}':{} for split in splits}
for split in splits:        # 全分割数ごとに
    positions = split ** 2  # 画面上の領域の個数。左上は 0 番で右下が positions-1 番
    for pos in range(positions):
        # 刺激画面上の各位置には _stimset を並べ替えた順番で個々の刺激が並ぶことになる。
        stims[f'split_{split}'][pos] = np.random.permutation(_stimset).tolist()

stims_info = []
for split in splits:
    positions = split ** 2  # 画面上の領域の個数。左上は 0 番で右下が positions-1 番
    for i in range(len(_stimset)):
        _tmp = []
        for pos in range(positions):
            _tmp.append((pos,stims[f'split_{split}'][pos][i])) 
        stims_info.append({'split':split, 'stim':_tmp})

print(f'len(stims_info:{len(stims_info)}')
print(f'stims_info[0]:{stims_info[0]}')

In [ ]:
def pt_get_original_img(stiminfo:dict=None,
                        img:PIL.Image=None,
                        symbols:list=bit.BIT(fontdata=notofonts).symbols,
                        verbose:bool=False):
    if img == None:
        img, canvas = make_canvas()
    else:
        canvas = PIL.ImageDraw.Draw(img)

    split = stiminfo['split']
    bboxes = []
    labels = []
    areas = make_div_areas(div=split)
    
    for stim in stiminfo['stim']:
        pos = stim[0]
        area = areas[pos]

        symbol  = stim[1][0]
        
        label   = symbols.index(symbol)
        labels.append(label)
        
        color    = stim[1][1]
        fontname = stim[1][2]
        fontsize = fontsizes[split]['size']
        font = fonts_info[f'{fontsize}_{fontname}']

        if symbol == '<line>' or symbol == '<background>':
            #print(symbol, color, fontname, area, fontsize, fontname, type(font))
            ;
        else:
            offset_x = ((area[2] - area[0]) - fontsize) >> 1
            offset_y = (area[3] - area[1] - fontsize) >> 1
            xy = (area[0]+offset_x, area[1]+offset_y)
            canvas.text(xy=xy,
                        text=symbol,
                        fill=color,
                        font=font,
                        anchor='lt')
            bbox = canvas.textbbox(xy=xy,
                                   text=symbol,
                                   font=font,
                                   anchor='lt',
                                   stroke_width=4)
            bboxes.append(bbox)
            if verbose:
                canvas.rectangle(xy=bbox, fill=None, outline='red', width=2)

    # ここから下は PyTorch 用変換
    pt_img = torch.Tensor(np.array(img)).permute(2,0,1)
    pt_labels = torch.as_tensor(labels, dtype=torch.int64)
        
    # convert boxes into a torch.Tensor
    pt_bboxes = torch.as_tensor(bboxes, dtype=torch.float32)

    # getting the areas of the boxes
    pt_area = (pt_bboxes[:, 3] - pt_bboxes[:, 1]) * (pt_bboxes[:, 2] - pt_bboxes[:, 0])
    pt_iscrowd = torch.zeros((pt_bboxes.shape[0],), dtype=torch.int64)

    pt_target = {}
    #pt_target["img"]     = pt_img
    pt_target["boxes"]   = pt_bboxes
    pt_target["labels"]  = pt_labels
    pt_target["area"]    = pt_area
    pt_target["iscrowd"] = pt_iscrowd
                
    return img, bboxes, pt_target

## 1.3 PyTorch 用データセットの作成

In [ ]:
class BIT_LineCancellation_dataset(torch.utils.data.Dataset):
    """留意事項
    1. データセットはタプルを返す。1 つ目の要素は画像の形状，2 つ目の要素は辞書である。
    2. 画像はデータセット定義時に指定したサイズでカラーモードは RGB
    3. 画像には 4 つのバウンディングボックスがあり，これはボックス内の 4 つのリストとラベルの長さから明らかである。
    """
    def __init__(self, 
                 stim_info:list=stims_info,
                 symbols:list = bit.BIT(fontdata=notofonts).symbols):
        
        super().__init__()
        self.symbols = symbols
        self.stim_info = stims_info
        
    def __get_original_img(self,
                           index:int):
        stiminfo = self.stim_info[index]
        img, bboxes, pt_target = pt_get_original_img(stiminfo)
        return img
        
    def __getitem__(self, 
                    index:int):
        stiminfo = self.stim_info[index]
        _img, bboxes, pt_target = pt_get_original_img(stiminfo)
        img = torch.Tensor(np.array(_img)/255.).permute(2,0,1)
        pt_target['image_id'] = torch.tensor([index])

        return img, pt_target

    def __len__(self):
        return len(self.stim_info)

bit_dataset = BIT_LineCancellation_dataset()

# 以下検証用
print(f'bit_dataset.__len__():{bit_dataset.__len__()}')
N = np.random.choice(bit_dataset.__len__())
img, labels = bit_dataset.__getitem__(N)

print(f'N:{N}')
print(f'labels:{labels}')
print([(_label, symbols[_label]) for _label in labels['labels']])

#_img = img.detach().numpy().transpose(1,2,0).clip(0,1)
#plt.subplot(1,2,1);plt.imshow(_img);plt.title('データ')
#plt.subplot(1,2,2);plt.imshow(pt_get_original_img(stims_info[N])[0]);plt.title('オリジナル')

## 1.4 データセットを分割して，訓練データとテストデータを作成

In [ ]:
# データセットを分割して，訓練データとテストデータを作成
N = bit_dataset.__len__()
N_train = int((N / 10) * 9)
N_test = N - N_train
print(f'N_train:{N_train}, N_test:{N_test}')
seed=42
train_dataset, test_dataset = torch.utils.data.random_split(bit_dataset, 
                                                            [N_train, N_test], 
                                                            generator=torch.Generator().manual_seed(seed))

## 1.5 データローダの作成

In [ ]:
# データローダの作成
def collate_fn(batch):
    return tuple(zip(*batch))

torch.manual_seed(42)
# 学習・検証用データローダの定義
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=128, 
    shuffle=True, 
    num_workers=0,
    collate_fn=collate_fn)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=128, 
    shuffle=False, 
    num_workers=0,
    collate_fn=collate_fn)

print(f'len(train_dataset):{len(train_dataset)}, len(test_dataset):{len(test_dataset)}')

# 2. モデルの定義と頭部の付け替え

In [ ]:
# モデルの定義と頭部の付け替え
import torch
import torchvision 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

#_bit = bit.BIT()

def get_object_detection_model(
    num_classes:int=1024)->torch.nn.Module:

    # MS-COCO で事前に学習させたモデルを読み込み
    # https://arxiv.org/abs/1506.01497
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT')
    #model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='FasterRCNN_ResNet50_FPN_Weights.DEFAULT')
    
    # 分類器の入力特徴数の取得
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    print(f'変換前 model.roi_heads:{model.roi_heads}')

    # 事前学習済頭部を新しいものに置き換え
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
    print(f'変換後 model.roi_heads:{model.roi_heads}')

    return model

# 上で定義した自作ヘルパ関数を使ってモデルを宣言
num_classes = len(symbols)
model = get_object_detection_model(num_classes)
model.roi_heads

## 3.1 事前訓練済パラメータの読み込み

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# データの ID を入れて，データを入手
download = drive.CreateFile({'id': '1nRM2YzkRakEExDoc42_Mtnrbw8pxRhb8'})
# https://drive.google.com/file/d/1nRM2YzkRakEExDoc42_Mtnrbw8pxRhb8/view?usp=sharing
download.GetContentFile('2022_1109letter_cancellation_0.pt')


In [ ]:
# 訓練済モデルがあれば読み込む
import os
#fname_model_trained = '2022_0620fine_tuned_bit_line_bisection.cpt'
#if os.path.exists(fname_model_trained):
#    XXX = torch.load(fname_model_trained)['model']

#fname_model_trained = '2022_1026line_cancellation_19.pt'
#fname_model_trained = '2022_1102letter_cancellation_2.pt'
fname_model_trained = '2022_1109letter_cancellation_0.pt'
if os.path.exists(fname_model_trained):
    XXX = torch.load(fname_model_trained)

model.load_state_dict(XXX)

In [ ]:
from bit_utils import torch_to_pil
from bit_utils import plot_img_bbox
from bit_utils import apply_nms

import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 3. 予測


In [ ]:
num = np.random.choice(test_dataset.__len__())
img, target = test_dataset.__getitem__(num)
img_orig = img.detach().numpy().transpose(1,2,0).clip(0,1)

model.eval()  # モデルを eval() モードに設定する。学習しないように
with torch.no_grad():
    prediction = model([img.to(device)])[0]
    
plot_img_bbox(img_orig,
              target, 
              title="グランドトルース",
              figsize=(3,3))

nms_prediction = apply_nms(prediction, iou_thresh=0.01)
plot_img_bbox(img.numpy().transpose(1,2,0), # .clip(0,1),
              nms_prediction, 
              title="モデル予測",
              figsize=(3,3))

In [ ]:
for k in nms_prediction.keys():
    print(k, type(nms_prediction[k]))
for i, box in enumerate(nms_prediction['boxes']):
    print(i, box.size(), type(box))

In [ ]:
#print(dir(nms_prediction))
c = nms_prediction.copy()
#help(nms_prediction)

_c = {'boxes':None, 'labels':None, 'scores':None}
for k in nms_prediction.keys():
    for box in nms_prediction['boxes']:
        left, top, right, bottom = box
        center = (left + right) / 2
        #print(left,top,right,bottom)